# Desafio - Semana 08

###### Resolvido por Hugo Malta em 01/08/2020

## Detalhes

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna `NU_NOTA_MT`) de quem participou do ENEM 2016. 

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `NU_NOTA_MT`.

---

In [37]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [38]:
pd.options.display.float_format = '{:,.2f}'.format

dados_train = pd.read_csv("train.csv")
dados_train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.00,...,5.00,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.00,...,nan,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.00,...,nan,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.00,...,5.00,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.00,...,nan,A,A,A,A,A,A,A,A,A


In [39]:
dados_test = pd.read_csv("test.csv")
dados_test.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,41,PR,22,F,3,1,1,5,1,...,40.00,420.00,B,A,C,A,A,C,C,A
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,21,MA,26,F,3,1,1,8,1,...,100.00,580.00,E,B,C,B,B,B,F,A
2,b38a03232f43b11c9d0788abaf060f7366053b6d,23,CE,21,M,1,1,2,0,2,...,80.00,320.00,E,E,D,B,B,A,NaN,A
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,15,PA,27,F,3,1,1,8,1,...,nan,nan,H,E,G,B,B,A,NaN,A
4,715494628a50142ce8cb17191cfe6d0f3cae0934,41,PR,18,M,1,1,2,0,2,...,0.00,320.00,D,H,H,C,B,A,NaN,A


Selecionando nos dados de treino as features que existem nos dados de teste

In [40]:
features_test = list(dados_test.columns)
features_test.append('NU_NOTA_MT')
#dados_train = dados_train[features_test].dropna()
dados_train[features_test]

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,NU_NOTA_MT
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,43,RS,24,M,1,1,1,4,1,...,520.00,D,D,C,A,A,C,H,A,399.40
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,23,CE,17,F,3,1,2,0,2,...,580.00,A,A,B,A,A,A,NaN,A,459.80
2,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,23,CE,21,F,3,1,3,0,1,...,nan,D,D,C,A,A,A,NaN,A,nan
3,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,33,RJ,25,F,0,1,1,9,1,...,nan,H,E,E,C,B,C,F,D,nan
4,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,13,AM,28,M,2,1,1,4,1,...,nan,E,D,C,A,A,B,F,A,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,aadf671ef8c1c6efa42e69432093ab83c17a52ac,35,SP,20,M,1,1,1,3,1,...,440.00,E,E,G,B,B,B,C,A,403.20
13726,461f7ef2753d9d5fa0b054930d6b0e0eec318e81,26,PE,33,F,3,2,1,10,1,...,560.00,A,A,C,A,B,B,I,A,452.40
13727,5354503700ecf2388f68990435390b6e1ad454ee,43,RS,15,F,1,1,3,0,1,...,600.00,C,G,B,A,B,A,NaN,A,398.00
13728,c7a9e94a42bd76b7978f13c013a61702e36cc42c,33,RJ,36,M,3,1,4,0,1,...,460.00,B,D,G,B,B,C,E,A,386.60


Verificando a correlação das features para escolha

In [41]:
corr = dados_train.corr()['NU_NOTA_MT'].sort_values(ascending=False)
corr.head(20)

NU_NOTA_MT                1.00
NU_NOTA_CN                0.58
NU_NOTA_CH                0.53
NU_NOTA_LC                0.49
NU_NOTA_REDACAO           0.38
TP_DEPENDENCIA_ADM_ESC    0.37
NU_NOTA_COMP3             0.35
NU_NOTA_COMP5             0.34
NU_NOTA_COMP4             0.34
NU_NOTA_COMP2             0.34
NU_NOTA_COMP1             0.30
CO_MUNICIPIO_NASCIMENTO   0.11
CO_UF_NASCIMENTO          0.11
CO_MUNICIPIO_PROVA        0.11
CO_MUNICIPIO_RESIDENCIA   0.11
CO_UF_PROVA               0.11
CO_UF_RESIDENCIA          0.11
CO_MUNICIPIO_ESC          0.10
CO_ESCOLA                 0.10
CO_UF_ESC                 0.10
Name: NU_NOTA_MT, dtype: float64

In [42]:
features = list(corr.head(11).index)
features.remove('NU_NOTA_MT')
features.remove('TP_DEPENDENCIA_ADM_ESC')
dados_train[features]

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO,NU_NOTA_COMP3,NU_NOTA_COMP5,NU_NOTA_COMP4,NU_NOTA_COMP2,NU_NOTA_COMP1
0,436.30,495.40,581.20,520.00,120.00,80.00,80.00,120.00,120.00
1,474.50,544.10,599.00,580.00,120.00,80.00,120.00,120.00,140.00
2,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...
13725,438.20,419.70,394.50,440.00,80.00,40.00,80.00,120.00,120.00
13726,423.80,557.90,533.00,560.00,120.00,80.00,120.00,120.00,120.00
13727,460.50,528.90,569.30,600.00,120.00,120.00,120.00,120.00,120.00
13728,422.50,621.70,569.00,460.00,80.00,100.00,80.00,100.00,100.00


In [48]:
x_treino = dados_train[features]
x_treino = x_treino.fillna(0)

In [49]:
y_treino = dados_train['NU_NOTA_MT']
y_treino = y_treino.fillna(0)

In [50]:
#modelo = LinearSVR(max_iter=10000)
#modelo.fit(x_treino, y_treino)
pipe = Pipeline(steps = #[('preencher_zero', SimpleImputer(strategy = 'constant')),
                        [('padronizar', StandardScaler()),
                        ('linearRegression', LinearRegression())])
pipe.fit(x_treino, y_treino)

Pipeline(memory=None,
         steps=[('padronizar',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearRegression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

Até aqui, o modelo está treinado.

Agora vou separar os dados para predição e aplicar.

In [52]:
x_teste = dados_test[features]
x_teste = x_teste.fillna(0)

In [53]:
y_teste = pipe.predict(x_teste)
y_teste[:5]

array([424.76057263, 482.58967419, 563.46394177,  -4.43766956,
       548.09016247])

Enfim, preciso juntar o y_teste à coluna de inscrição.

In [54]:
x_teste = x_teste.assign(NU_NOTA_MT = y_teste)
x_teste.head()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO,NU_NOTA_COMP3,NU_NOTA_COMP5,NU_NOTA_COMP4,NU_NOTA_COMP2,NU_NOTA_COMP1,NU_NOTA_MT
0,464.80,443.50,431.80,420.00,80.00,40.00,100.00,80.00,120.00,424.76
1,391.10,491.10,548.00,580.00,120.00,100.00,120.00,120.00,120.00,482.59
2,595.90,622.70,613.60,320.00,40.00,80.00,80.00,40.00,80.00,563.46
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-4.44
4,592.90,492.60,571.40,320.00,60.00,0.00,80.00,80.00,100.00,548.09


In [55]:
dados_test_nu_inscr = dados_test['NU_INSCRICAO']
dados_test_nu_inscr.head()

0    73ff9fcc02f0a99919906c942c2e1a1042cdcf98
1    71a95f9f1b91a82c65ad94abbdf9f54e6066f968
2    b38a03232f43b11c9d0788abaf060f7366053b6d
3    70b682d9a3636be23f6120fa9d6b164eb3c6002d
4    715494628a50142ce8cb17191cfe6d0f3cae0934
Name: NU_INSCRICAO, dtype: object

In [56]:
x_teste = x_teste.join(dados_test_nu_inscr)
x_teste.head()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO,NU_NOTA_COMP3,NU_NOTA_COMP5,NU_NOTA_COMP4,NU_NOTA_COMP2,NU_NOTA_COMP1,NU_NOTA_MT,NU_INSCRICAO
0,464.80,443.50,431.80,420.00,80.00,40.00,100.00,80.00,120.00,424.76,73ff9fcc02f0a99919906c942c2e1a1042cdcf98
1,391.10,491.10,548.00,580.00,120.00,100.00,120.00,120.00,120.00,482.59,71a95f9f1b91a82c65ad94abbdf9f54e6066f968
2,595.90,622.70,613.60,320.00,40.00,80.00,80.00,40.00,80.00,563.46,b38a03232f43b11c9d0788abaf060f7366053b6d
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-4.44,70b682d9a3636be23f6120fa9d6b164eb3c6002d
4,592.90,492.60,571.40,320.00,60.00,0.00,80.00,80.00,100.00,548.09,715494628a50142ce8cb17191cfe6d0f3cae0934


In [57]:
resultado = x_teste[['NU_INSCRICAO','NU_NOTA_MT']]
resultado

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,424.76
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,482.59
2,b38a03232f43b11c9d0788abaf060f7366053b6d,563.46
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,-4.44
4,715494628a50142ce8cb17191cfe6d0f3cae0934,548.09
...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,444.47
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,429.79
4573,655fa6306720ff16e825903b5422a46608a77545,594.05
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,489.95


In [58]:
resultado.to_csv('answer.csv', index=False)

In [ ]:
resultado